In [1]:
import pickle
path = r'./IEMOCAP_emotion_cause_features.pkl'

In [2]:
loaded = pickle.load(open(path, 'rb'), encoding='latin1')

videoIDs, videoSpeakers, videoLabels, causeLabels, causeLabels2, causeLabels3, videoText,\
videoAudio, videoVisual, videoSentence, trainVid,\
testVid = loaded

'''
label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5} 
# disgust가 없음, fear가 없고 frustration이 있음
'''

"\nlabel index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5} \n# disgust가 없음, fear가 없고 frustration이 있음\n"

In [3]:
videoSpeakers

{'Ses03M_impro08b': ['M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'F',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'M',
  'F',
  'M',
  'F',
  'M',
  'M',
  'M',
  'M',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F'],
 'Ses05M_impro02': ['M',
  'M',
  'M',
  'F',
  'M',
  'M',
  'M',
  'M',
  'F',
  'M',
  'F',
  'F',
  'M',
  'F',
  'M',
  'M',
  'M',
  'M',
  'M',
  'M',
  'M',
  'F',
  'M',
  'F',
  'M',
  'M',
  'M',
  'F',
  'F'],
 'Ses05M_impro01': ['F',
  'F',
  'F',
  'F',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'F',
  'F',
  'M',
  'F',
  'F',
  'M',
  'F',
  'M',
  'F',
  'F',
  'M',
  'F',
  'M',
  'F',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'M',
  'F',
  'M',
  'F',
  'M'],
 'Ses03M_impro08a': ['M',
  'F',
  'M',
  'F',
  'F',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'M',
  'F',
  'F',
  'M',
  'M',
  'F',
  'M',

In [3]:
def get_dataset_dictionary(vids):
    label_mapping = {0: 'happy', 1: 'sad', 2: 'neutral', 3: 'angry', 4: 'excited', 5: 'frustrated'}
    # label_mapping = {0: 'happy', 1: 'sad', 2: 'neutral', 3: 'angry', 4: 'fear', 5: 'disgust'}
    # # 사실은 fear로 표시된 건 excited고, disgust로 표시되는 건 frustrated인데, 기존 프리프로세싱이랑 겹치니까 안 겹치도록 저렇게 마킹함
    full_dataset = {}
    key_list  = list(vids)
    for key in key_list:
        a_dialog = []
        turn_num = 1
        dialog_full_utterances = [_ for _ in videoSentence[key]]
        for speaker, sentence, emotion, cl1, cl2, cl3 in zip(videoSpeakers[key], videoSentence[key], videoLabels[key], causeLabels[key], causeLabels2[key], causeLabels3[key]):
            a_utterance = {}
            
            a_utterance['turn'] = turn_num
            a_utterance['speaker'] = "A" if speaker=="M" else "B"
            a_utterance['utterance'] = sentence
            a_utterance['emotion'] = label_mapping[emotion]
            
            cl_list = [cl1, cl2, cl3]
            cl_nonzero = [cl for cl in cl_list if cl != 0]
            
            if a_utterance['emotion'] != 'neutral':
                a_utterance['expanded emotion cause span'] = [dialog_full_utterances[_-1] for _ in cl_nonzero]
                cl_nonzero = ['b' if value>turn_num else value for value in cl_nonzero]
                a_utterance['expanded emotion cause evidence'] = cl_nonzero
                a_utterance['type'] = [
                    'no_context' for _ in cl_nonzero
                ]
            turn_num += 1
            a_dialog.append(a_utterance)
            
        full_dataset[key] = [a_dialog]

    return full_dataset

In [11]:
import random # trainVid의 20%를 validation set으로 사용
random.seed(42)
train_vid_list = list(trainVid)
# train_vid_list를 shuffle
train_vid_shuffled = random.sample(train_vid_list, len(train_vid_list))

# 120 크기의 train_vid_shuffled를 96:24로 나눠서 5개의 cross-fold를 만들어보자
trainVid_fold = []
validVid_fold = []

# train_vid_shuffled를 5개의 cross-fold로 나눠서 trainVid_fold, validVid_fold 생성하기 (5개의 vid 리스트)
for i in range(5):
    valid_start_index = i*24
    valid_end_index = (i+1)*24
    
    # train_vid_list[valid_start_index]부터 train_vid_list[valid_end_index]까지 validVid_fold에 넣고, 나머지는 trainVid_fold에 넣기
    validVid_fold.append(train_vid_shuffled[valid_start_index:valid_end_index])
    trainVid_fold.append(train_vid_shuffled[0:valid_start_index] + train_vid_shuffled[valid_end_index:])   

'''
output: Vid list
    trainVid_fold = [96, 96, 96, 96, 96]
    validVid_fold = [24, 24, 24, 24, 24]
'''


'\noutput: Vid list\n    trainVid_fold = [96, 96, 96, 96, 96]\n    validVid_fold = [24, 24, 24, 24, 24]\n'

In [14]:
import json
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


# 각각의 trainVid_fold, validVid_fold로부터 json 형태의 파일 3개 생성, pkl 형태 파일 1개 생성
for i, (trainVid_, validVid_) in enumerate(zip(trainVid_fold, validVid_fold)):
    print(f'fold {i} processing...: \n\t({len(trainVid_)}){trainVid_}\n\t({len(validVid_)}){validVid_}')
    
    to_pickle = videoIDs, videoSpeakers, videoLabels, causeLabels, causeLabels2, causeLabels3, videoText,\
    videoAudio, videoVisual, videoSentence, set(trainVid_),\
    testVid, set(validVid_)
    
    pickle.dump(to_pickle, open(f'./ConvECPE_fold_{i}.pkl', 'wb'))
    
    # train_json
    train_json = get_dataset_dictionary(trainVid_)
    with open(f'./ConvECPE_fold_{i}_train.json', 'w') as f:
        json.dump(train_json, f, indent="\t", cls=NpEncoder)
    # valid_json
    valid_json = get_dataset_dictionary(validVid_)
    with open(f'./ConvECPE_fold_{i}_valid.json', 'w') as f:
        json.dump(valid_json, f, indent="\t", cls=NpEncoder)
    # test_json
    test_json = get_dataset_dictionary(testVid)
    with open(f'./ConvECPE_fold_{i}_test.json', 'w') as f:
        json.dump(test_json, f, indent="\t", cls=NpEncoder)
        
    

fold 0 processing...: 
	(96)['Ses01F_script02_2', 'Ses02M_impro03', 'Ses03M_script03_2', 'Ses02M_script03_2', 'Ses04F_script03_2', 'Ses02M_impro07', 'Ses03F_impro06', 'Ses02M_impro05', 'Ses04F_impro03', 'Ses03M_impro04', 'Ses02M_impro06', 'Ses02F_impro06', 'Ses04F_impro06', 'Ses02F_script01_2', 'Ses01M_script03_1', 'Ses04F_impro02', 'Ses04F_impro08', 'Ses01F_impro02', 'Ses03F_script02_1', 'Ses02F_impro02', 'Ses01F_script01_2', 'Ses03F_impro08', 'Ses01M_script02_1', 'Ses03F_script01_2', 'Ses02M_script03_1', 'Ses03F_script03_1', 'Ses03M_impro05b', 'Ses01M_script01_3', 'Ses03M_script02_2', 'Ses04M_impro08', 'Ses01F_impro04', 'Ses01M_script02_2', 'Ses04F_impro01', 'Ses03M_impro08a', 'Ses01F_impro03', 'Ses01F_script03_1', 'Ses03M_script01_3', 'Ses01F_script01_3', 'Ses03F_impro02', 'Ses03F_script01_1', 'Ses04F_impro05', 'Ses03M_impro03', 'Ses01M_impro07', 'Ses02M_impro08', 'Ses04M_impro02', 'Ses04F_script03_1', 'Ses04F_script01_2', 'Ses02F_script02_1', 'Ses02F_script01_3', 'Ses04M_script01_3

In [6]:



Vids = trainVid
full_dataset_train = get_dataset_dictionary(Vids)
json.dump(full_dataset_train, open('ConvECPE_train_originalsplit.json', 'w'), indent="\t", cls=NpEncoder)


Vids = testVid
full_dataset_test = get_dataset_dictionary(Vids)
json.dump(full_dataset_test, open('ConvECPE_test_originalsplit.json', 'w'), indent="\t", cls=NpEncoder)

# ConvECPE_vids_dict.json을 set type으로 train, test, valid 불러오기
import json
import numpy as np
vid_dict = json.load(open('ConvECPE_vids_dict.json', 'r'))

train_sets = set(vid_dict['train'])
test_sets = set(vid_dict['test'])
valid_sets = set(vid_dict['valid'])

Vids = train_sets
full_dataset_train = get_dataset_dictionary(Vids)
json.dump(full_dataset_train, open('ConvECPE_train.json', 'w'), indent="\t", cls=NpEncoder)

Vids = test_sets
full_dataset_test = get_dataset_dictionary(Vids)
json.dump(full_dataset_test, open('ConvECPE_test.json', 'w'), indent="\t", cls=NpEncoder)

Vids = valid_sets
full_dataset_test = get_dataset_dictionary(Vids)
json.dump(full_dataset_test, open('ConvECPE_valid.json', 'w'), indent="\t", cls=NpEncoder)

In [20]:
# ConvECPE_vids_dict.json을 set type으로 train, test, valid 불러오기
import json
import numpy as np
vid_dict = json.load(open('ConvECPE_vids_dict.json', 'r'))

train_sets = set(vid_dict['train'])
test_sets = set(vid_dict['test'])
valid_sets = set(vid_dict['valid'])

Vids = train_sets
full_dataset_train = get_dataset_dictionary(Vids)
json.dump(full_dataset_train, open('ConvECPE_train.json', 'w'), indent="\t", cls=NpEncoder)

Vids = test_sets
full_dataset_test = get_dataset_dictionary(Vids)
json.dump(full_dataset_test, open('ConvECPE_test.json', 'w'), indent="\t", cls=NpEncoder)

Vids = valid_sets
full_dataset_test = get_dataset_dictionary(Vids)
json.dump(full_dataset_test, open('ConvECPE_valid.json', 'w'), indent="\t", cls=NpEncoder)

In [14]:
vids_dict = {
    'train': list(train_sets),
    'valid': list(valid_sets),
    'test': list(test_sets)
}

# vids_dict를 json으로 저장
json.dump(vids_dict, open('ConvECPE_vids_dict-spare.json', 'w'), indent="\t", cls=NpEncoder)


In [18]:
# ConvECPE_vids_dict.json을 set type으로 train, test, valid 불러오기
import json
import numpy as np
vid_dict = json.load(open('ConvECPE_vids_dict.json', 'r'))

train_sets = set(vid_dict['train'])
test_sets = set(vid_dict['test'])
valid_sets = set(vid_dict['valid'])